In [1]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[0.5717],
        [0.6152]], grad_fn=<AddmmBackward0>)

In [2]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[ 0.1498,  0.3229,  0.2434,  0.2390, -0.2272, -0.0408,  0.0337,  0.3360]])), ('bias', tensor([-0.0060]))])


In [3]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.0060], requires_grad=True)
tensor([-0.0060])


In [4]:
net[2].weight.grad == None

True

In [5]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [9]:
net.state_dict()['2.bias'].data

tensor([-0.0060])

In [10]:
def block():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())


def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block_{i}', block())
    return net

In [11]:
rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.4244],
        [-0.4243]], grad_fn=<AddmmBackward0>)

In [12]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block_0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block_1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block_2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block_3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [13]:
print(rgnet[0][1][0].bias)

Parameter containing:
tensor([ 0.4916,  0.0111,  0.2787, -0.4871, -0.1629, -0.2732, -0.4183,  0.3634],
       requires_grad=True)


In [14]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0., std=0.01)
        nn.init.zeros_(m.bias)


net.apply(init_normal)

Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=1, bias=True)
)

In [15]:
net[0].weight.data[0], net[0].bias.data[0]

(tensor([0.0066, 0.0039, 0.0077, 0.0164]), tensor(0.))

In [16]:
net[0].weight.data

tensor([[ 0.0066,  0.0039,  0.0077,  0.0164],
        [-0.0130, -0.0003, -0.0055, -0.0063],
        [-0.0089,  0.0140, -0.0053, -0.0059],
        [-0.0008, -0.0102, -0.0064,  0.0078],
        [-0.0024,  0.0016, -0.0023,  0.0083],
        [ 0.0141, -0.0080, -0.0004, -0.0114],
        [-0.0076, -0.0069, -0.0085,  0.0278],
        [-0.0005, -0.0002,  0.0130,  0.0018]])

In [17]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1.)
        nn.init.zeros_(m.bias)


net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [18]:
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)


def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42.)

In [19]:
net

Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=1, bias=True)
)

In [20]:
net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([0.3820, 0.4358, 0.0297, 0.1496])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [22]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape) for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5


net.apply(my_init)

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=1, bias=True)
)

In [25]:
for i in net[0].named_parameters():
    print(i)

('weight', Parameter containing:
tensor([[ 5.0640,  6.1774,  0.0000,  0.0000],
        [-9.1384,  0.0000, -0.0000,  0.0000],
        [ 0.0000,  5.4794, -8.1074,  0.0000],
        [-5.1532, -5.2492, -0.0000,  6.6662],
        [ 7.8388,  8.2080,  8.8168, -6.4682],
        [ 0.0000, -7.8916,  0.0000, -0.0000],
        [ 6.2385, -0.0000,  5.1035, -0.0000],
        [ 0.0000,  0.0000, -8.6914,  0.0000]], requires_grad=True))
('bias', Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0.], requires_grad=True))


In [26]:
net[0].weight[:2]

tensor([[ 5.0640,  6.1774,  0.0000,  0.0000],
        [-9.1384,  0.0000, -0.0000,  0.0000]], grad_fn=<SliceBackward0>)

In [27]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000,  7.1774,  1.0000,  1.0000])

In [28]:
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(X)

tensor([[-0.1815],
        [-0.1806]], grad_fn=<AddmmBackward0>)

In [29]:
net

Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=8, bias=True)
  (3): ReLU()
  (4): Linear(in_features=8, out_features=8, bias=True)
  (5): ReLU()
  (6): Linear(in_features=8, out_features=1, bias=True)
)

In [30]:
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])


In [31]:
net[2].weight.data[0, 0] = 100
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
